### 数据收集


In [109]:
import pandas as pd
import pymysql
db = pymysql.connect('mysql-backup.dian.so','yihui','GMYAvsgzE9fRRC1r','oss_view') # 打开数据库连接
sql='''
select e.id as '用户id', nick_name as '花名',
e.sex as '性别',
education_first as '学历',
r.birth_date as '生日',
e.level_id as '级别' 
,recruite_channel as '来源' ,
e.department_id  as '部门id-1',
(
select name from agent_department ad where id=e.department_id
)as '部门名称-1',
(
select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=e.department_id 
)
)as '部门名称-2',
(
select id from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=e.department_id 
)
)as '部门id-2',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=e.department_id 
  )
 )
)as '部门名称-3',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=e.department_id 
    )
  )
 )
)as '部门名称-4',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=e.department_id 

          )
    )
  )
 )
)as '部门名称-5',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=e.department_id 
                      )

          )
    )
  )
 )
)as '部门名称-6',
(
  select name from agent_department ad2 where id =(
  select parent_id from agent_department ad3 where id=(
    select parent_id from agent_department ad4 where id=(
          select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                      select parent_id from agent_department ad4 where id=(
                                            select parent_id from agent_department ad4 where id=e.department_id 

                      )
                      )

          )
    )
  )
 )
)as '部门名称-7'

from  hr_resume as r left JOIN  agent_employee as e
on e.id=r.employee_id
where  real_status=1 and status=0 and agent_id=1
order by e.id desc
'''

user=pd.read_sql(sql=sql,con=db)
db.close()

In [110]:
user.head(3)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7
0,24061,森金,1,高中,1998-01-06,2,None,2087,湘潭小分队BDM_众乐,株洲,1602,湖南分区,西南区,直营事业部,商务发展部,杭州小电科技股份有限公司
1,24059,徐宁,1,大专,1992-05-10,2,None,1134,徐州BDM_史进,徐州,971,苏北分区,华东区,直营事业部,商务发展部,杭州小电科技股份有限公司
2,24058,尚儒,1,大专,1997-06-23,2,None,2968,杭州BDM_星燎,杭州二区,2097,杭州分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司


### 数据处理

In [105]:
"""
时间转换成年代
"""
def get_birth_day(birth_date):
    dt95='1995-01-01'
    dt90='1990-01-01'
    dt85='1985-01-01'
    dt80='1980-01-01'

    birth=str(birth_date)
    if birth>dt95:
        return '95后'
    if birth<dt95 and birth>dt90:
        return '90-95'
    if birth>dt85 and birth<dt90:
        return '85-90'
    if birth>dt80 and birth<dt85:
        return '80-85'
    if birth<dt80:
        return '70后'
    return '未知'

"""
将性别标记位转换成文字
"""
def get_sex(sex):
    if sex==1:
        return '男'
    else:
        return '女'


In [111]:
user['性别']=user.apply(lambda x: get_sex(x['性别']),axis=1)


In [114]:
user['年龄段']=user.apply(lambda x: get_birth_day(x['生日']),axis=1)


In [118]:
user.head(20)

,用户id,花名,性别,学历,生日,级别,来源,部门id-1,部门名称-1,部门名称-2,部门id-2,部门名称-3,部门名称-4,部门名称-5,部门名称-6,部门名称-7,年龄段
0,24061,森金,男,高中,1998-01-06,2,None,2087,湘潭小分队BDM_众乐,株洲,1602,湖南分区,西南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
1,24059,徐宁,男,大专,1992-05-10,2,None,1134,徐州BDM_史进,徐州,971,苏北分区,华东区,直营事业部,商务发展部,杭州小电科技股份有限公司,90-95
2,24058,尚儒,男,大专,1997-06-23,2,None,2968,杭州BDM_星燎,杭州二区,2097,杭州分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
3,24057,久茶,男,本科,1996-06-20,2,None,1288,武汉BDM_老炮,武汉三区,2359,长汉分区,CBD南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
4,24056,桃鸢,男,中专,1995-02-07,2,None,1244,鞍山小分队虚拟BDM,辽宁外区,3086,黑吉辽分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
5,24055,安遥,女,大专,1988-01-20,2,None,2350,绥化小分队BDM_云溪,黑吉外区,3083,黑吉辽分区,华北区,直营事业部,商务发展部,杭州小电科技股份有限公司,85-90
6,24053,狂签,男,高中,1990-09-10,2,None,219,郑州BDM_能者,郑州一区,178,郑州分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,90-95
7,24052,左枝,女,大专,2001-10-09,2,None,1282,衡阳BDM_乔森,衡阳,1266,湖南分区,西南区,直营事业部,商务发展部,杭州小电科技股份有限公司,95后
8,24051,星神,男,高中,1991-10-01,2,None,3180,成都BDM_七万,成都三区,2393,成都分区,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,90-95
9,24050,子长,男,大专,1993-12-01,2,None,2777,西安BDM_冰海,西安三区,2685,CBD北区,直营事业部,商务发展部,杭州小电科技股份有限公司,None,90-95


### 数据探索

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns

user.groupby('性别').size()


性别
1    4186
2     921
dtype: int64

In [121]:
biz_sex_pie=user[ (user['部门名称-2']=='商务发展部') | (user['部门名称-3']=='商务发展部')| (user['部门名称-4']=='商务发展部')| (user['部门名称-5']=='商务发展部')| (user['部门名称-6']=='商务发展部')| (user['部门名称-7']=='商务发展部')].groupby('性别').size()

biz_sex_pie

性别
女     481
男    3554
dtype: int64

In [122]:
doy_sex_pie=user[ (user['部门名称-2']=='产品研发中心') | (user['部门名称-3']=='产品研发中心')| (user['部门名称-4']=='产品研发中心')| (user['部门名称-5']=='产品研发中心')| (user['部门名称-6']=='产品研发中心')| (user['部门名称-7']=='产品研发中心')].groupby('性别').size()
doy_sex_pie

性别
女     37
男    186
dtype: int64

In [123]:
user[ (user['部门名称-2']=='测试质量部') | (user['部门名称-3']=='测试质量部')| (user['部门名称-4']=='测试质量部')| (user['部门名称-5']=='测试质量部')| (user['部门名称-6']=='测试质量部')| (user['部门名称-7']=='测试质量部')].groupby('性别').size()

性别
女    10
男    15
dtype: int64

In [73]:
doy_pie=user[ (user['部门名称-2']=='产品研发中心') | (user['部门名称-3']=='产品研发中心')| (user['部门名称-4']=='产品研发中心')| (user['部门名称-5']=='产品研发中心')| (user['部门名称-6']=='产品研发中心')| (user['部门名称-7']=='产品研发中心')].groupby('学历').size()
doy_pie

学历
中专         1
大专        31
本科       168
硕士研究生     22
高中         1
dtype: int64

In [70]:
biz_pie=user[ (user['部门名称-2']=='商务发展部') | (user['部门名称-3']=='商务发展部')| (user['部门名称-4']=='商务发展部')| (user['部门名称-5']=='商务发展部')| (user['部门名称-6']=='商务发展部')| (user['部门名称-7']=='商务发展部')].groupby('学历').size()

biz_pie

学历
中专        482
初中         32
大专       2387
小学          1
本科        663
硕士研究生       2
高中        469
dtype: int64

In [46]:
user.groupby('学历').size()

学历
MBA         4
中专        537
初中         38
大专       2770
小学          1
本科       1177
硕士研究生      44
高中        536
dtype: int64

In [128]:
age_pie=user.groupby('年龄段').size()
age_pie

年龄段
70后        18
80-85     148
85-90     864
90-95    2352
95后      1722
dtype: int64

In [61]:
a=user.groupby('学历').size()
type(a)
a.index.tolist()
a.values.tolist()

[4, 537, 38, 2770, 1, 1177, 44, 536]

### 数据展现

In [66]:
def user_pie(title_str,lables_str,values_str):
    import plotly.offline as py
    from plotly.graph_objs import Scatter, Layout
    import plotly.graph_objs as go

    py.init_notebook_mode(connected=True)

    pyplt = py.offline.plot
    labels = lables_str
    values = values_str
    trace = [go.Pie(labels=labels, values=values)]
    layout = go.Layout(
        title = title_str,
    )
    figure = go.Figure(data = trace, layout = layout)
    py.iplot(figure)


In [130]:
user_pie('小电整体学历分布比例',a.index.tolist(),a.values.tolist())

In [71]:
user_pie('小电商务发展部学历分布比例',biz_pie.index.tolist(),biz_pie.values.tolist())

In [74]:
user_pie('小电产品研发部学历分布比例',doy_pie.index.tolist(),doy_pie.values.tolist())

In [124]:
user_pie('小电产品研发部性别分布比例',doy_sex_pie.index.tolist(),doy_sex_pie.values.tolist())

In [125]:
user_pie('小电商务发展部性别分布比例',biz_sex_pie.index.tolist(),biz_sex_pie.values.tolist())

In [129]:
user_pie('小电年龄段分布',age_pie.index.tolist(),age_pie.values.tolist())

### 报告撰写

#### 学历
- 小电整体学历大专，本科是主流
- 但是部门之间差异比较大
  1. 在商务发展部，大专是主流
  2. 在产品研发部，本科和硕士是主流
#### 性别
  - 小电是个男多女少的公司，想找妹子的需要避开
#### 年龄
  - 小电整体年龄结构比较年轻，90后是绝对主流